# LinkupSearchTool

> [Linkup](https://www.linkup.so/) 提供了一个 API，用于将 LLM 连接到网络以及 Linkup Premium Partner 资源。

本笔记本提供了 LinkupSearchTool [工具](/docs/concepts/tools/) 的快速入门指南。如需了解 LinkupSearchTool 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/linkup/tools/linkup_langchain.search_tool.LinkupSearchTool.html)。

## 概述

### 集成详情

| Class | Package | Serializable | [JS 支持](https://js.langchain.com/docs/integrations/tools/linkup_search) |  Package 最新版本 |
| :--- | :--- | :---: | :---: | :---: |
| [LinkupSearchTool](https://python.langchain.com/api_reference/linkup/tools/linkup_langchain.search_tool.LinkupSearchTool.html) | [langchain-linkup](https://python.langchain.com/api_reference/linkup/index.html) | ❌ | ❌ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-linkup?style=flat-square&label=%20) |

## 设置

要使用 Linkup 提供商，您需要一个有效的 API 密钥，您可以在此[注册](https://app.linkup.so/sign-up)后找到。要运行以下示例，您还需要一个 OpenAI API 密钥。

### 安装

此工具位于 `langchain-linkup` 包中：

In [ ]:
%pip install -qU langchain-linkup

### 凭据

In [ ]:
import getpass
import os

# if not os.environ.get("LINKUP_API_KEY"):
#     os.environ["LINKUP_API_KEY"] = getpass.getpass("LINKUP API key:\n")

设置 [LangSmith](https://smith.langchain.com/) 以获得一流的可观测性也很有帮助（但不是必需的）：

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## 实例化

此处我们展示了如何实例化一个 LinkupSearchTool 工具实例，其中

In [ ]:
from langchain_linkup import LinkupSearchTool

tool = LinkupSearchTool(
    depth="deep",  # "standard" or "deep"
    output_type="searchResults",  # "searchResults", "sourcedAnswer" or "structured"
    linkup_api_key=None,  # API key can be passed here or set as the LINKUP_API_KEY environment variable
)

## 调用

### 直接使用参数调用

该工具仅接受一个 `query` 参数，它是一个字符串。

In [10]:
tool.invoke({"query": "Who won the latest US presidential elections?"})

LinkupSearchResults(results=[LinkupSearchResult(name='US presidential election results 2024: Harris vs. Trump | Live maps ...', url='https://www.reuters.com/graphics/USA-ELECTION/RESULTS/zjpqnemxwvx/', content='Updated results from the 2024 election for the US president. Reuters live coverage of the 2024 US President, Senate, House and state governors races.'), LinkupSearchResult(name='Election 2024: Presidential results - CNN', url='https://www.cnn.com/election/2024/results/president', content='View maps and real-time results for the 2024 US presidential election matchup between former President Donald Trump and Vice President Kamala Harris. For more ...'), LinkupSearchResult(name='Presidential Election 2024 Live Results: Donald Trump wins - NBC News', url='https://www.nbcnews.com/politics/2024-elections/president-results', content='View live election results from the 2024 presidential race as Kamala Harris and Donald Trump face off. See the map of votes by state as results are tallie

### 使用 ToolCall 调用

我们也可以使用模型生成的 ToolCall 来调用工具，在这种情况下将返回一个 ToolMessage：

In [11]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "Who won the latest US presidential elections?"},
    "id": "1",
    "name": tool.name,
    "type": "tool_call",
}
tool.invoke(model_generated_tool_call)

ToolMessage(content='results=[LinkupSearchResult(name=\'US presidential election results 2024: Harris vs. Trump | Live maps ...\', url=\'https://www.reuters.com/graphics/USA-ELECTION/RESULTS/zjpqnemxwvx/\', content=\'Updated results from the 2024 election for the US president. Reuters live coverage of the 2024 US President, Senate, House and state governors races.\'), LinkupSearchResult(name=\'Election 2024: Presidential results - CNN\', url=\'https://www.cnn.com/election/2024/results/president\', content=\'View maps and real-time results for the 2024 US presidential election matchup between former President Donald Trump and Vice President Kamala Harris. For more ...\'), LinkupSearchResult(name=\'Presidential Election 2024 Live Results: Donald Trump wins - NBC News\', url=\'https://www.nbcnews.com/politics/2024-elections/president-results\', content=\'View live election results from the 2024 presidential race as Kamala Harris and Donald Trump face off. See the map of votes by state as 

## 链式调用

我们可以将我们的工具进行链式调用，首先将其绑定到一个 [工具调用模型](/docs/how_to/tool_calling/)，然后进行调用：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [ ]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai")

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("Who won the 2016 US presidential elections?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JcHj0XLARWRnwrrLhUoBjOV1', 'function': {'arguments': '{"query":"2016 US presidential election winner"}', 'name': 'linkup'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 1037, 'total_tokens': 1047, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_831e067d82', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd7642ed-4509-4c96-8934-20bd0b986c3f-0', tool_calls=[{'name': 'linkup', 'args': {'query': '2016 US presidential election winner'}, 'id': 'call_JcHj0XLARWRnwrrLhUoBjOV1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1037, 'output_tokens': 10, 'total_tokens': 1047, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

## API 参考

LinkupSearchTool 的所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/linkup/tools/linkup_langchain.search_tool.LinkupSearchTool.html)。